In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import Packages.ClusteringHelper as ch
from Packages.TimeEvolving import DataEvolver
from textdistance import DamerauLevenshtein, Levenshtein
import numpy as np
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from collections import Counter

In [42]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
ents_data = data[data['entities'] != ''].copy()

In [43]:
ents_data = ch.add_entities_embedding(ents_data,
                                      "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings")
ents_data_filtered = ch.filter_data(ents_data, 3)

In [44]:
documents = set(ents_data.documents)

In [45]:
gold_entities = set()
evolving = DataEvolver(documents, ents_data, randomly=False, step=20)
total_current_entities = []
iterator = evolving.__iter__()
_ = iterator.__next__()

In [46]:
current_mentions = list(evolving.get_current_data().mentions)
def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])  # extract indices
    if len(current_mentions[i]) < 4:
        if current_mentions[i] == current_mentions[j]:
            return 0
        else:
            return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
    else:
        return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())

def dam_lev_metric(x, y):
    i, j = int(x[0]), int(y[0])  # extract indices
    if len(current_mentions[i]) < 4:
        if current_mentions[i] == current_mentions[j]:
            return 0
        else:
            return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
    else:
        return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())

X = np.arange(len(current_mentions)).reshape(-1, 1)
clusterizator2 = DBSCAN(metric=lev_metric, eps=1, min_samples=0, n_jobs=-1)
clusters = clusterizator2.fit_predict(X)

In [47]:
current_encodings = evolving.get_current_data()['encodings'].values
current_entities = evolving.get_current_data()['entities'].values
cee_dict = {k: {'entities': [], 'mentions': [], 'encodings': [], 'sotto_clusters': None} for k in set(clusters)}
for i, cluster in enumerate(clusters):
    cee_dict[cluster]['entities'].append(current_entities[i])
    cee_dict[cluster]['mentions'].append(current_mentions[i])
    cee_dict[cluster]['encodings'].append(current_encodings[i])
cee_list = cee_dict.values()
# cee_list

In [48]:
clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.035, linkage="single")
for cluster in cee_dict.keys():
    try:
        cee_dict[cluster]['sotto_clusters'] = clusterizator.fit_predict(cee_dict[cluster]['encodings'])
    except ValueError:
        cee_dict[cluster]['sotto_clusters'] = np.zeros(1, dtype=np.int8)

In [49]:
for x in cee_list:
    print("mentions:" , x['mentions'])
    print("entities:" , x['entities'])
    print("sotto_clusters:", x['sotto_clusters'])
    print("-------")
    break

mentions: ['German', 'German', 'Germany', 'German', 'Germany', 'Germany', 'German', 'German', 'Germans', 'German', 'German', 'GERMAN']
entities: ['Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany', 'Germany']
sotto_clusters: [0 0 0 0 0 0 0 0 0 0 0 0]
-------


In [50]:
cee_sottocluser_list = []
for el in cee_list:
    sotto_cluster = {k: {'entities': [], 'mentions': [], 'encodings': []} for k in set(el['sotto_clusters'])}
    for i, key in enumerate(el['sotto_clusters']):
        sotto_cluster[key]['mentions'].append(el['mentions'][i])
        sotto_cluster[key]['entities'].append(el['entities'][i])
        sotto_cluster[key]['encodings'].append(el['encodings'][i])
    cee_sottocluser_list.append(sotto_cluster)

In [51]:
cee_sottocluser_list = [clusters_dict[key] for clusters_dict in cee_sottocluser_list for key in clusters_dict ]

In [52]:
sotto_encodings = [np.mean(cluster['encodings'], axis=0) for cluster in cee_sottocluser_list]

In [53]:
clusterizator3 = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.015, linkage="single")
cluster_sotto_cluster = clusterizator3.fit_predict(sotto_encodings)

In [54]:
final_clusters = {k:Counter() for k in set(cluster_sotto_cluster)}
for i, x in enumerate(cee_sottocluser_list):
    final_clusters[cluster_sotto_cluster[i]] =  final_clusters[cluster_sotto_cluster[i]] + Counter(x['entities'])

In [73]:
gold_entities = set.union(gold_entities, set(current_entities))
total_current_entities = total_current_entities + current_entities

CEAFm

In [75]:
# precision
best_alignment = ch.get_optimal_alignment(final_clusters, gold_entities, is_dict=True)

In [80]:
best_alignment

In [81]:
CEAFm_f1 = sum(best_alignment.values()) / len(total_current_entities)
print(CEAFm_f1)


0.8664921465968587
